In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('../'))

import numpy as np
from matplotlib import pyplot as plt
import arrayfire as af
af.set_backend('cpu')

from dg_maxwell import utils
from dg_maxwell import lagrange

plt.rcParams['figure.figsize']     = 12, 7.5
plt.rcParams['lines.linewidth']    = 1.5
plt.rcParams['font.family']        = 'serif'
plt.rcParams['font.weight']        = 'bold'
plt.rcParams['font.size']          = 20  
plt.rcParams['font.sans-serif']    = 'serif'
plt.rcParams['text.usetex']        = True
plt.rcParams['axes.linewidth']     = 1.5
plt.rcParams['axes.titlesize']     = 'medium'
plt.rcParams['axes.labelsize']     = 'medium'

plt.rcParams['xtick.major.size']   = 8
plt.rcParams['xtick.minor.size']   = 4
plt.rcParams['xtick.major.pad']    = 8
plt.rcParams['xtick.minor.pad']    = 8
plt.rcParams['xtick.color']        = 'k'
plt.rcParams['xtick.labelsize']    = 'medium'
plt.rcParams['xtick.direction']    = 'in'    

plt.rcParams['ytick.major.size']   = 8
plt.rcParams['ytick.minor.size']   = 4
plt.rcParams['ytick.major.pad']    = 8
plt.rcParams['ytick.minor.pad']    = 8
plt.rcParams['ytick.color']        = 'k'
plt.rcParams['ytick.labelsize']    = 'medium'
plt.rcParams['ytick.direction']    = 'in'
plt.rcParams['text.usetex']        = True
plt.rcParams['text.latex.unicode'] = True


In [38]:
N_LGL = 8
xi_LGL = lagrange.LGL_points(N_LGL)
L_basis_poly1d, L_basis_af = lagrange.lagrange_polynomials(xi_LGL)
L_basis_af = af.np_to_af_array(L_basis_af)

In [39]:
for poly in L_basis_poly1d:
    print(poly)

        7         6         5         4         3         2
-3.352 x + 3.352 x + 3.867 x - 3.867 x - 1.055 x + 1.055 x + 0.03906 x - 0.03906
       7         6         5         4         3         2
8.141 x - 7.097 x - 11.35 x + 9.892 x + 3.332 x - 2.904 x - 0.1249 x + 0.1088
        7         6         5         4        3        2
-10.36 x + 6.129 x + 18.68 x - 11.05 x - 8.67 x + 5.13 x + 0.3448 x - 0.204
       7         6         5        4         3        2
11.39 x - 2.384 x - 24.03 x + 5.03 x + 15.67 x - 3.28 x - 3.03 x + 0.6343
        7         6         5        4         3        2
-11.39 x - 2.384 x + 24.03 x + 5.03 x - 15.67 x - 3.28 x + 3.03 x + 0.6343
       7         6         5         4        3        2
10.36 x + 6.129 x - 18.68 x - 11.05 x + 8.67 x + 5.13 x - 0.3448 x - 0.204
        7         6         5         4         3         2
-8.141 x - 7.097 x + 11.35 x + 9.892 x - 3.332 x - 2.904 x + 0.1249 x + 0.1088
       7         6         5         4         3     

In [40]:
print(L_basis_af)

arrayfire.Array()
Type: double

[8 8 1 1]
   -3.3516     3.3516     3.8672    -3.8672    -1.0547     1.0547     0.0391    -0.0391 
    8.1407    -7.0966   -11.3475     9.8921     3.3316    -2.9043    -0.1249     0.1088 
  -10.3581     6.1289    18.6834   -11.0549    -8.6700     5.1301     0.3448    -0.2040 
   11.3898    -2.3839   -24.0330     5.0301    15.6735    -3.2805    -3.0304     0.6343 
  -11.3898    -2.3839    24.0330     5.0301   -15.6735    -3.2805     3.0304     0.6343 
   10.3581     6.1289   -18.6834   -11.0549     8.6700     5.1301    -0.3448    -0.2040 
   -8.1407    -7.0966    11.3475     9.8921    -3.3316    -2.9043     0.1249     0.1088 
    3.3516     3.3516    -3.8672    -3.8672     1.0547     1.0547    -0.0391    -0.0391 




In [41]:
def Li_basis_value(L_basis, i, xi):
    '''
    Finds the value of the :math:`i^{th}` Lagrange basis polynomial
    at the given :math:`\\xi` coordinates.
    
    Parameters
    ----------
    L_basis : af.Array [N_LGL N_LGL 1 1]
              Lagrange basis polynomial coefficient array
              
    i       : af.array [N 1 1 1]
              Index of the Lagrange basis polynomials
              to be evaluated.
              
    xi      : af.Array [N 1 1 1]
              :math:`\\xi` coordinates at which the :math:`i^{th}` Lagrange
              basis polynomial is to be evaluated.
    '''
    N_LGL = int(L_basis.shape[0])
    xi_   = af.tile(af.transpose(xi), d0 = N_LGL)
    power = af.tile(af.flip(af.np_to_af_array(np.arange(N_LGL)), dim = 0), d0 = 1, d1 = xi.shape[0])
    
    xi_power = xi_**power
    
    return af.matmul(L_basis[i], xi_power)

In [42]:
Li_basis_value(L_basis_af, i, af.np_to_af_array(np.linspace(-1., 1, 10)))

arrayfire.Array()
Type: double

[8 10 1 1]
   -1.0000    -0.1722    -0.0163    -0.0005    -0.0000     0.0000     0.0005     0.0163     0.1722     1.0000 
    1.0000     0.2214     0.0294     0.0014     0.0000     0.0000     0.0014     0.0294     0.2214     1.0000 
   -1.0000    -0.2846    -0.0529    -0.0041    -0.0000     0.0000     0.0041     0.0529     0.2846     1.0000 
    1.0000     0.3660     0.0953     0.0123     0.0002     0.0002     0.0123     0.0953     0.3660     1.0000 
   -1.0000    -0.4705    -0.1715    -0.0370    -0.0014     0.0014     0.0370     0.1715     0.4705     1.0000 
    1.0000     0.6049     0.3086     0.1111     0.0123     0.0123     0.1111     0.3086     0.6049     1.0000 
   -1.0000    -0.7778    -0.5556    -0.3333    -0.1111     0.1111     0.3333     0.5556     0.7778     1.0000 
    1.0000     1.0000     1.0000     1.0000     1.0000     1.0000     1.0000     1.0000     1.0000     1.0000 


arrayfire.Array()
Type: double

[3 10 1 1]
    1.0000    -0.1321   

In [18]:
i = af.np_to_af_array(np.arange(3, dtype = np.int32))
powers = af.range(N_LGL, dtype = af.Dtype.u16)

In [15]:
print(x)

arrayfire.Array()
Type: int

[3 1 1 1]
         0 
         1 
         2 




In [14]:
L_basis_af[x]

array([[ -3.3515625 ,   3.3515625 ,   3.8671875 ,  -3.8671875 ,
         -1.0546875 ,   1.0546875 ,   0.0390625 ,  -0.0390625 ],
       [  8.14072272,  -7.09659483, -11.34747768,   9.89205188,
          3.33160871,  -2.90429707,  -0.12485375,   0.10884002],
       [-10.35813683,   6.12891144,  18.68335516, -11.05494464,
         -8.67003714,   5.13006255,   0.34481881,  -0.20402935]])

In [12]:
def lagrange_function_value(lagrange_coeff_array, xi):
    '''
    '''
    xi_tile    = af.transpose(af.tile(xi, 1, xi.shape[0]))
    power      = af.flip(af.range(xi.shape[0]))
    power_tile = af.tile(power, 1, xi.shape[0])
    xi_pow     = af.arith.pow(xi_tile, power_tile)
    index      = af.range(xi.shape[0])
    print(lagrange_coeff_array)
    print(xi_pow)
    L_i        = af.blas.matmul(lagrange_coeff_array[index], xi_pow)
    
    return L_i

In [14]:
lagrange.lagrange_function_value(L_basis_af)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices